#OPEN-SOURCE MODEL TESTING
This notebook tests the various open-source models on a custom dataset using RAG Prompting and Zero-Shot Prompting

##How to run
Run the sections:

 - Import

 - Data Loading

 - Retrieval Model

 - Functions

To import the required dependencies and load the query_data.Load the retrieval model. Run the functions for zero-shot prompts and rag prompts.

Test the model of your choice



# Import

In [ ]:
!pip install openai faiss-cpu sentence-transformers ctransformers[cuda]

In [ ]:
import json
import openai
import tqdm
from transformers import AutoModel
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import tqdm
import time
from ctransformers import AutoModelForCausalLM
import pickle

# Data Loading

In [ ]:
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1kkkmxgPF0EHfBLx79Epyo4STdiA-HmRX' -O examples.json

In [ ]:
import json

In [ ]:
!wget -O tools.json "https://drive.usercontent.google.com/download?id=1RSC_94KKmvtWnI15RVZkLCy4eTVhxGU_&export=download&authuser=0&confirm=t&uuid=394a0e2f-bb02-4027-8224-ef451c182f49&at=APZUnTVfd2focJ6u3DYRiDeKWBUr:1702403341998"
!wget -O examples.json "https://drive.usercontent.google.com/download?id=1n9eede9tNiqfPfWyc3lx1AHgqJgxLFQ5&export=download&authuser=0&confirm=t&uuid=eabc4504-aca7-4dfc-a6d9-52d357149ee6&at=APZUnTWmVVzr0utprDNGk8VXTU6f:1702403392202"
!wget -O claude_examples_ps_tools_all.json "https://drive.usercontent.google.com/download?id=17orzUW3_n31pZFyL0F-ekGH5iKlh2SUX&export=download&authuser=0&confirm=t&uuid=61097f63-24fe-4c51-9710-77d7fd204c2b&at=APZUnTXUxVA_067GuWmlaIrm74Io:1702403397617"
with open('/content/examples.json', 'r') as f:
    query_data = json.load(f)

with open('/content/tools.json', 'r') as f:
    tool_data = json.load(f)

with open('/content/claude_examples_ps_tools_all.json', 'r') as f:
    tool_query_data = json.load(f)

In [ ]:
query_1 = "Summarize my P1 issues in triage"
query_2 = "Prioritize tasks similar to deve/0:issue/5"
query_3 = "Add my blocker tickets to the current sprint"
query_4 = "Prioritize low tickets that need response"
query_5 = "Get all issues with priority p0 or p1 assigned to me and summarize them"
query_6 = "List all tickets from github that need a response and add them to the current sprint"
query_7 = "Get tasks assigned to me in triage, summarize them and create issues from that"
query_8 = "List high severity github tickets from customer ABC, summarize them and prioritize the summary"
query_9 = "Get all blocker tickets assigned to me, summarize and create issues from summary"
query_10 = "List issues assigned to John, summarize them and prioritize"
query_11 = "List high priority tickets from Acme Corp, summarize them"
query_12 = "Add my actionable tasks from meeting notes M to my current sprint"
query_13 = "What are similar issues to issue ISSUE-1?"
query_14 = "Summarize my tickets needing response"
query_15 = "Get me all the blocker tickets from github and slack channels under parts ENH-123, PROD-123, CAPL-123, CAPL-359 that need a response, prioritize them and add them to the current sprint"
query_16 = "What are all my p0, p1 priority issues under parts HGH-262, FEAT-007 in the design stage? Summarize them and add them to the current sprint"
query_17 = "Get all issues assigned to current user that are in design stage under parts FEAT-123 and FEAT-456. Filter them by priority p1 and p2. Summarize them and add them to current sprint."
query_18 = "Get all tasks assigned to me. Filter tasks in progress stage. Summarize the tasks. Get similar tasks to the top 3 tasks from the summary and add them to the current sprint"
query_19 = "Search for user DEV-123, prioritize and add their P0 issues to the current sprint"
query_20 = "Get the status of a build from last night"
query_21 = "Schedule a meeting with the stakeholders"

queries_list_eval = [
    query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9,
    query_10, query_11, query_12, query_13, query_14, query_15, query_16, query_17,
    query_18, query_19, query_20, query_21
]


# Retrieval Model

In [ ]:
queries=[]
for inp in query_data:
  queries.append(inp['query'])

In [ ]:
queries

In [ ]:
cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))

In [ ]:
# model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

In [ ]:
query_embeddings = model.encode(queries)

In [ ]:
d = query_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(query_embeddings)

In [ ]:
def get_relevant_queries(user_query,query_data,encoder=model,k=3):
  user_query_embedding = model.encode(user_query)
  user_query_embedding=np.expand_dims(user_query_embedding,axis=0)
  D, I = index.search(user_query_embedding, k)
  relevant_queries=[]
  for i in range(k):
    relevant_queries_index = I[0][i]
    relevant_queries.append(query_data[relevant_queries_index])
  return relevant_queries

# Functions

In [ ]:
def prompt(relevant_queries,k=3):
  l=[]
  for i in range(k):
    l.append({
                "role": "user",
                "content": f"QUESTION: {relevant_queries[i]['query']}",
            })
    l.append({
                "role": "assistant",
                "content": f"ANSWER: {relevant_queries[i]['answer']}",
            })
  return l

In [ ]:
def chat_prompt_simple(tools,query_data, question):
    sys_prompt = '''
                You are a highly intelligent AI assistant that can help users with questions about a certain wide range of topics involving customer relationship management (CRM). You specialize in writing API calls that are correct, simple, and concise.
                {{
                "tools": {}
                }}
                EXAMPLES
              '''.format(tools)
    return [
          {

              "role": "system",
              "content": sys_prompt

          },
          {
              "role": "user",
              "content": "QUESTION: What are all my issues in the triage stage under part FEAT-123? Summarize them.",
          },
          {
              "role": "assistant",
              "content":
              """
              {
              'REASONING': 'First, we use the "whoami" tool to get the user id of the person asking the query. Then, we use the "works" list tool to figure out the work items assigned to the user. We pass  $$PREV[0] to get the previous tool call's output as the "owned_by" argument. We pass the "stage_name" argument as "triage" to get issues in the triage stage, and "applies_to_part" argument as "FEAT-123" to filter out the relevant parts. We then use the "summarize_objects" tool with $$PREV[1] as its argument to summarize the outputs of the last tool call.'
              'ANSWER:':
              [
              {
              "tool_name": "whoami",
              "arguments": []
              },
              {
              "tool_name": "works_list",
              "arguments": [
              {
              "argument_name": "stage.name",
              "argument_value": "triage"
              },
              {
              "argument_name": "applies_to_part",
              "argument_value": "FEAT-123"
              },
              {
              "argument_name": "owned_by",
              "argument_value": "$$PREV[0]"
              }
              ]
              },
              {
              "tool_name": "summarize_objects",
              "arguments": [
              {
              "argument_name": "objects",
              "argument_value": "$$PREV[1]"
              }
              ]
              }
              ]
              }
              """
          },
          {
              "role": "user",
              "content": "QUESTION: Summarize high severity tickets from the customer UltimateCustomer"
          },
          {
              "role": "assistant",
              "content": """
              {'REASONING' : 'First, we use the "search_objects_by_name" tool to search for the exact user, and filter them out by passing the query as "UltimateCustomer", to only get the user id of 'UltimateCustomer'. We then search for the issues created by the customer by using the "works_list" tool with the previous output by passing $$PREV[0] as 'ticket.rev_org' to filter out the issues by the customer, and pass the 'ticket.severity' arg as 'high' .We then use the "summarize_objects" tool to finally summarize these issues, by passing in the outputs of the previous tool call as $$PREV[1].'
              'ANSWER' :
              [
              {
              "tool_name": "search_object_by_name",
              "arguments": [
              {
              "argument_name": "query",
              "argument_value": "UltimateCustomer"
              }
              ]
              },
              {
              "tool_name": "works_list",
              "arguments": [
              {
              "argument_name": "ticket.rev_org",
              "argument_value": "$$PREV[0]"
              },
              {
              "argument_name": "ticket.severity",
              "argument_value": "["high"]"
              }
              ]
              },
              {
              "tool_name": "summarize_objects",
              "arguments": [
              {
              "argument_name": "objects",
              "argument_value": "$$PREV[1]"
              }
              ]
              }
              ]
              }
              """},
      {
        "role": "user",
        "content": "QUESTION: Get all work items similar to TKT-123, summarize them, create issues from that summary, and prioritize them."
      },
      {
        "role": "assistant",
        "content": """
        {
        'REASONING': 'We use the tool "get_similar_work_items" to get work items similar to "TKT-123". We then summarize the outputs with the "summarize_objects" tool, passing the previous tool's outputs in the 'objects' argument as $$PREV[0]. We then use "create_actionable_tasks_from_text" on the outputs of the previous tool call with $$PREV[1], as the previous tool call returned the output numbered 1,  to generate issues from the summarized work items. Finally, we use the "prioritize_objects" tool with the argument 'objects' passed as $$PREV[2]  to prioritize the outputs of the last tool call.'
        'ANSWER': [
        {
        "tool_name": "get_similar_work_items",
        "arguments": [
        {
        "argument_name": "work_id",
        "argument_value": "TKT-123"
        }
        ]
        },
        {
        "tool_name": "summarize_objects",
        "arguments": [
        {
        "argument_name": "objects",
        "argument_value": "$$PREV[0]"
        }
        ]
        },
        {
        "tool_name": "create_actionable_tasks_from_text",
        "arguments": [
        {
        "argument_name": "text",
        "argument_value": "$$PREV[1]"
        }
        ]
        },
        {
        "tool_name": "prioritize_objects",
        "arguments": [
        {
        "argument_name": "objects",
        "argument_value": "$$PREV[2]"
        }
        ]
        }
        ]
        }
        """},
      {
        "role": "system",
        "content": "Now, it is your task to answer the following query"
      },
      {
        "role": "user",
        "content": f"QUESTION: {question}"
      }
      ]

In [ ]:
def chat_prompt_with_retrieval(tools,query_data,question):
    sys_prompt = '''
                You are a highly intelligent AI assistant that can help users with questions about a certain wide range of topics involving customer relationship management (CRM). You specialize in writing API calls that are correct, simple, and concise.
                {{
                "tools": {}
                }}
                EXAMPLES
              '''.format(tools)
    relevant_queries=get_relevant_queries(question,query_data)
    return [
          {
              "role": "system",
              "content": sys_prompt,
          },
          {
              "role": "user",
              "content": f"QUESTION: {relevant_queries[0]['query']}",
          },
          {
              "role": "assistant",
              "content": f"{relevant_queries[0]['answer']}",
          },
          {
              "role": "user",
              "content": f"QUESTION: {relevant_queries[1]['query']}",
          },
          {
              "role": "assistant",
              "content":f"{relevant_queries[1]['answer']}",
          },
          {
              "role": "user",
              "content": f"QUESTION: {relevant_queries[2]['query']}",
          },
          {
              "role": "assistant",
              "content": f"{relevant_queries[2]['answer']}",
          },
          {
              "role": "system",
              "content": "Now, it is your task to answer the following query and the answer should be in the same format as mentioned answer above and it should be on json and solve answer step by step with reasoning and no need to give your reasoning while giving the answer. The output should be in json format and use double quotes ("") instead of single quotes ('')"
          },
          {
              "role": "user",
              "content": f"QUESTION: {question}"
          }
      ]

In [ ]:
def retrieve_prompt_with_model(tool_data,query_data,query,sys_start,sys_end,usr_start,usr_end,ast_start):
  relevant_queries=get_relevant_queries(query,query_data)
  prompt_template=f"""{sys_start} You are a highly intelligent AI assistant that can help users with questions that can be answered by the use of a set of TOOLS. You specialize in identifying tools with arguments to sove the query.
  You are given the following TOOLS with their DESCRIPTIONS, ARGUMENTS to the tool in question with their DESCRIPTIONS.
  {tool_data}.
   To reference the value of the ith tool in the chain, use $$PREV[i] as argument value, where i refers to the output of the tool positioned in the ith place in the JSON.
   EXAMPLE QUERY 1: {relevant_queries[0]['query']}
   ANSWER: {relevant_queries[0]['answer']}
   EXAMPLE QUERY 2: {relevant_queries[1]['query']}
   ANSWER: {relevant_queries[1]['answer']}
   EXAMPLE QUERY 3: {relevant_queries[2]['query']}
   ANSWER: {relevant_queries[2]['answer']}
  The above are solutions to some queries.
   {sys_end}. {usr_start} It is your task to answer the following QUERY. Think step by step. ONLY OUTPUT THE ANSWER TO THE QUERY AND THE ANSWER SHOULD BE IN JSON FORMAT. DO NOT DEVIATE from the instructions and keep in mind that the output answer should be in correct json format.
   QUERY: {query}. {usr_end} {ast_start}"""
  return prompt_template

In [ ]:
def predict(tool_data,query_data,test_query,llm,temperature,sys_start,sys_end,usr_start,usr_end,ast_start):
  return llm(retrieve_prompt_with_model(tool_data,query_data,test_query,sys_start,sys_end,usr_start,usr_end,ast_start))

In [ ]:
def get_json(pred):
  return json.loads(pred[pred.find('{'):-1*("".join(reversed(pred)).find('}')+1)] + '}')

In [ ]:
def postprocess(pred, tool_data):
  json_pred = get_json(pred)
  tools = {}
  for i,tool in enumerate(tool_data["tools"]):
    tools[tool["tool_name"]] = tool
    tools[tool["tool_name"]]["args"] = {}
    for arg in tool["argument_list"]:
      tools[tool["tool_name"]]["args"][arg["argument_name"]] = arg
  for tool in json_pred:
    for j, arg in enumerate(tool["argument_list"]):
      arg_type = tools[tool["tool_name"]]["args"][arg["argument_name"]]["argument_type"]
      temp = json_pred[i]["argument_list"][j]["argument_value"]
      if arg_type == "string":
        temp = str(temp)
      elif arg_type == "array of strings" or arg_type == "array of objects":
        temp = [str(temp)]
      json_pred[i]["argument_list"][j]["argument_value"] = temp
  return json_pred

#RAG Prompting

##Zephyr

In [ ]:
zephyr = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-beta-GGUF", model_file="zephyr-7b-beta.Q4_K_M.gguf", model_type='mistral',gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
response_zephyr=[]
for query in tqdm.tqdm(queries):
  a=predict(tool_data,query_data,query,zephyr,0.5,'<|system|>','</s>','<|user|>','</s>','<|assistant|>')
  response_zephyr.append({'query':query,'answer':a})

In [ ]:
response_zephyr

## Mistral

In [ ]:
mistral = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral",gpu_layers=900,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
response_mistral=[]
for query in tqdm.tqdm(queries_list_eval):
  print(query)
  a=predict(tool_data,query_data,query,mistral,0.5,'<s>[INST]',' ',' ','[/INST}</s>','')
  response_mistral.append({'query':query,'answer':a})

In [ ]:
response_mistral

## LLama 7B

In [ ]:
llama = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-GGUF", model_file="llama-2-7b.Q4_K_M.gguf", model_type="llama",gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
user_query='Find all the tasks owned by the current user that need a response and are associated with Rev organizations "REV-123" and "REV-456". Sort them by priority.'

In [ ]:
pred=predict(tool_data,query_data,user_query,llama,0.5,' ',' ',' ',' ','')
pred

In [ ]:
ans=[]
for inp in tqdm.tqdm(test_query_data):
  query=inp['query']
  a=predict(tool_data,query_data,query,llama,0.5,' ',' ',' ',' ','')
  ans.append({'query':query,'answer':a})

In [ ]:
ans

In [ ]:
with open('llama_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

In [ ]:
with open('./drive/MyDrive/Devrev Agent 007/Results/llama_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

## Neural chat

In [ ]:
neural_chat = AutoModelForCausalLM.from_pretrained("TheBloke/neural-chat-7B-v3-1-GGUF", model_file="neural-chat-7b-v3-1.Q4_K_M.gguf", model_type="mistral",gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
user_query='Find all the tasks owned by the current user that need a response and are associated with Rev organizations "REV-123" and "REV-456". Sort them by priority.'

In [ ]:
pred=predict(tool_data,query_data,user_query,neural_chat,0.5,'### System:',' ',' ','### User:','### Assistant:')
pred

In [ ]:
ans=[]
for inp in tqdm.tqdm(test_query_data):
  query=inp['query']
  a=predict(tool_data,query_data,query,neural_chat,0.5,'### System:',' ',' ','### User:','### Assistant:')
  ans.append({'query':query,'answer':a})

In [ ]:
ans

In [ ]:
with open('neural_chat_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

In [ ]:
with open('./drive/MyDrive/Devrev Agent 007/Results/neural_chat_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

## Orca mini 7B

In [ ]:
orca = AutoModelForCausalLM.from_pretrained("TheBloke/orca_mini_v3_7B-GGUF",gpu_layers=900,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
response_orca=[]
for query in tqdm.tqdm(queries_list_eval):
  print(query)
  a=predict(tool_data,query_data,"### Input: "+query,orca,0.5,'### System:',' ','### User:',' ','### Response:')
  response_orca.append({'query':query,'answer':a})

In [ ]:
response_orca

In [ ]:
with open('orca_7B_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

In [ ]:
with open('./drive/MyDrive/Devrev Agent 007/Results/orca_7B_rag_dset.pkl','wb') as f:
  pickle.dump(ans,f)

## YI-6B

In [ ]:
import torch

In [ ]:
yi_6b = AutoModelForCausalLM.from_pretrained("TheBloke/Yi-6B-GGUF", model_file="yi-6b.Q5_K_M.gguf", model_type="yi", gpu_layers=900,context_length=4096)

In [ ]:
response_yi6b=[]
for query in tqdm.tqdm(queries_list_eval):
  print(query)
  a=predict(tool_data,query_data,query,yi_6b,0.5,'','','','','')
  response_yi6b.append({'query':query,'answer':a})

In [ ]:
response_yi6b

In [ ]:
starling = AutoModelForCausalLM.from_pretrained("TheBloke/Starling-LM-7B-alpha-GGUF",model_file = "starling-lm-7b-alpha.Q5_K_S.gguf", gpu_layers=400,context_length=4096)

In [ ]:
pred=predict(tool_data,query_data,user_query,starling,0.5,'','','','','')
pred

In [ ]:
ans=[]
for inp in tqdm.tqdm(test_query_data):
  query=inp['query']
  a=predict(tool_data,query_data,""+query,llm,0.5,'','','','','')
  ans.append({'query':query,'answer':a})

## Neural Hermes

In [ ]:
neural_hermes = AutoModelForCausalLM.from_pretrained("TheBloke/NeuralHermes-2.5-Mistral-7B-GGUF",model_file = "neuralhermes-2.5-mistral-7b.Q5_K_M.gguf", gpu_layers=400,context_length=4096)

In [ ]:
pred=predict(tool_data,query_data,user_query,neural_hermes,0.5,'<|im_start|>system','<|im_end|>','<|im_start|>user','<|im_end|>','<|im_start>asisstant')
pred

In [ ]:
ans=[]
for inp in tqdm.tqdm(test_query_data):
  query=inp['query']
  a=predict(tool_data,query_data,query,neural_hermes,0.5,'<|im_start|>system','<|im_end|>','<|im_start|>user','<|im_end|>','<|im_start>asisstant')
  ans.append({'query':query,'answer':a})

ans

## Yi-6B-Dragon

In [ ]:
yi_34b = AutoModelForCausalLM.from_pretrained("TheBloke/dragon-yi-6B-v0-GGUF",model_file = "dragon-yi-6b-v0.Q5_K_M.gguf", gpu_layers=400,context_length=4096)

In [ ]:
ans=[]
for inp in tqdm.tqdm(test_query_data):
  query=inp['query']
  a=predict(tool_data,query_data,query,yi_34b,0.3,'','','','','')
  ans.append({'query':query,'answer':a})

ans

## Cybertron UNA

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip uninstall transformers

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

In [ ]:
import bitsandbytes
import transformers
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import tqdm
import torch

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_type=torch.float16)

In [ ]:
 tokenizer = AutoTokenizer.from_pretrained("fblgit/una-cybertron-7b-v2-bf16",trust_remote_code=True)
 cybertron = AutoModelForCausalLM.from_pretrained("fblgit/una-cybertron-7b-v2-bf16", device_map="cuda:0", trust_remote_code=True,quantization_config=quantization_config).eval()

In [ ]:
query_1 = "Summarize my P1 issues in triage"
query_2 = "Prioritize tasks similar to deve/0:issue/5"
query_3 = "Add my blocker tickets to the current sprint"
query_4 = "Prioritize low tickets that need response"
query_5 = "Get all issues with priority p0 or p1 assigned to me and summarize them"
query_6 = "List all tickets from github that need a response and add them to the current sprint"
query_7 = "Get tasks assigned to me in triage, summarize them and create issues from that"
query_8 = "List high severity github tickets from customer ABC, summarize them and prioritize the summary"
query_9 = "Get all blocker tickets assigned to me, summarize and create issues from summary"
query_10 = "List issues assigned to John, summarize them and prioritize"
query_11 = "List high priority tickets from Acme Corp, summarize them"
query_12 = "Add my actionable tasks from meeting notes M to my current sprint"
query_13 = "What are similar issues to issue ISSUE-1?"
query_14 = "Summarize my tickets needing response"
query_15 = "Get me all the blocker tickets from github and slack channels under parts ENH-123, PROD-123, CAPL-123, CAPL-359 that need a response, prioritize them and add them to the current sprint"
query_16 = "What are all my p0, p1 priority issues under parts HGH-262, FEAT-007 in the design stage? Summarize them and add them to the current sprint"
query_17 = "Get all issues assigned to current user that are in design stage under parts FEAT-123 and FEAT-456. Filter them by priority p1 and p2. Summarize them and add them to current sprint."
query_18 = "Get all tasks assigned to me. Filter tasks in progress stage. Summarize the tasks. Get similar tasks to the top 3 tasks from the summary and add them to the current sprint"
query_19 = "Search for user DEV-123, prioritize and add their P0 issues to the current sprint"
query_20 = "Get the status of a build from last night"
query_21 = "Schedule a meeting with the stakeholders"

queries = [
    query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9,
    query_10, query_11, query_12, query_13, query_14, query_15, query_16, query_17,
    query_18, query_19, query_20, query_21
]


In [ ]:
pp1=retrieve_prompt_with_model(tool_data,query_data,query_1,'<|im_start|>system','<|im_end|>','<|im_start|>user','<|im_end|>','<|im_start>assistant')

In [ ]:
input=tokenizer(pp1,return_tensors='pt')
response = tokenizer.decode(cybertron.generate(**input,max_new_tokens=4096)[0][len(input.input_ids[0]):],skip_special_tokens=True,)

In [ ]:
response_simple=[]
for query_ in tqdm.tqdm(queries):
  pp1 = retrieve_prompt_with_model(tool_data,query_data,query_,'<|im_start|>system','<|im_end|>','<|im_start|>user','<|im_end|>','<|im_start>assistant')
  input=tokenizer(pp1,return_tensors='pt')
  response = tokenizer.decode(cybertron.generate(**input,max_new_tokens=4096)[0][len(input.input_ids[0]):],skip_special_tokens=True)
  response_simple.append(response)

In [ ]:
response_simple

In [ ]:
for l in response_simple:
  print(l)
  print("\n")

In [ ]:
!pip install numba

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

#Zero shot Prompting

##Functions

In [ ]:
def create_prompt_zero_shot(user_query):
    message = [
        {
            "role": "system",
            "content": "You are a intelligent AI agent specialized in giving the tool responses given a dictionary of tools. Here is the dictionary of tools: "+ json.dumps(tool_data)
        }
    ]
    message.append({
        "role" : "user",
        "content" : '''
        Now its your task to respond to the user queries in the format given below
        FORMAT:[{"tool_name": "...", "arguments": [{"argument_name": "...", "argument_value": ... (depending on the argument_type)}, ...]}, ...]
        To reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool’s index in the array If the query could not be answered with the given set of tools, output an empty list instead.
        Output in the JSON format
        '''
    })
    message.append({
        "role" : "user",
        "content" : "Query: "+ user_query
    })
    return message

In [ ]:
ex ="""[{"tool_name": "...", "arguments": [{"argument_name": "...", "argument_value": ... (depending on the argument_type)}, ...]}]"""

In [ ]:
def retrieve_prompt_with_model(tool_data,query_data,query,sys_start,sys_end,usr_start,usr_end,ast_start):
  prompt_template=f"""{sys_start}You are a intelligent AI agent specialized in giving the tool responses given a dictionary of tools. Here is the dictionary of tools: {tool_data}
   {sys_end}. {usr_start} Now its your task to respond to the user queries in the format given below
        FORMAT:[{ex}, ...]
        To reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool’s index in the array If the query could not be answered with the given set of tools, output an empty list instead.
        Output in the JSON format
   QUERY: {query}. {usr_end} {ast_start}"""
  return prompt_template

In [ ]:
def predict(tool_data,query_data,test_query,llm,temperature,sys_start,sys_end,usr_start,usr_end,ast_start):
  return llm(retrieve_prompt_with_model(tool_data,query_data,test_query,sys_start,sys_end,usr_start,usr_end,ast_start))

In [ ]:
query_1 = "Summarize my P1 issues in triage"
query_2 = "Prioritize tasks similar to deve/0:issue/5"
query_3 = "Add my blocker tickets to the current sprint"
query_4 = "Prioritize low tickets that need response"
query_5 = "Get all issues with priority p0 or p1 assigned to me and summarize them"
query_6 = "List all tickets from github that need a response and add them to the current sprint"
query_7 = "Get tasks assigned to me in triage, summarize them and create issues from that"
query_8 = "List high severity github tickets from customer ABC, summarize them and prioritize the summary"
query_9 = "Get all blocker tickets assigned to me, summarize and create issues from summary"
query_10 = "List issues assigned to John, summarize them and prioritize"
query_11 = "List high priority tickets from Acme Corp, summarize them"
query_12 = "Add my actionable tasks from meeting notes M to my current sprint"
query_13 = "What are similar issues to issue ISSUE-1?"
query_14 = "Summarize my tickets needing response"
query_15 = "Get me all the blocker tickets from github and slack channels under parts ENH-123, PROD-123, CAPL-123, CAPL-359 that need a response, prioritize them and add them to the current sprint"
query_16 = "What are all my p0, p1 priority issues under parts HGH-262, FEAT-007 in the design stage? Summarize them and add them to the current sprint"
query_17 = "Get all issues assigned to current user that are in design stage under parts FEAT-123 and FEAT-456. Filter them by priority p1 and p2. Summarize them and add them to current sprint."
query_18 = "Get all tasks assigned to me. Filter tasks in progress stage. Summarize the tasks. Get similar tasks to the top 3 tasks from the summary and add them to the current sprint"
query_19 = "Search for user DEV-123, prioritize and add their P0 issues to the current sprint"
query_20 = "Get the status of a build from last night"
query_21 = "Schedule a meeting with the stakeholders"

queries = [
    query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9,
    query_10, query_11, query_12, query_13, query_14, query_15, query_16, query_17,
    query_18, query_19, query_20, query_21
]


##Zephyr

In [ ]:
zephyr = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-beta-GGUF", model_file="zephyr-7b-beta.Q4_K_M.gguf", model_type='mistral',gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
response_zephyr=[]
for query in tqdm.tqdm(queries):
  a=predict(tool_data,query_data,query,zephyr,0.5,'<|system|>','</s>','<|user|>','</s>','<|assistant|>')
  response_zephyr.append({'query':query,'answer':a})

In [ ]:
response_zephyr

##Neural chat

In [ ]:
neural_chat = AutoModelForCausalLM.from_pretrained("TheBloke/neural-chat-7B-v3-1-GGUF", model_file="neural-chat-7b-v3-1.Q4_K_M.gguf", model_type="mistral",gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
ans=[]
for inp in tqdm.tqdm(queries):

  a=predict(tool_data,query_data,inp,neural_chat,0.5,'### System:',' ',' ','### User:','### Assistant:')
  ans.append({'query':query,'answer':a})

In [ ]:
ans

##Mistral

In [ ]:
mistral = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral",gpu_layers=900,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
response_mistral=[]
for query in tqdm.tqdm(queries):
  print(query)
  a=predict(tool_data,query_data,query,mistral,0.5,'<s>[INST]',' ',' ','[/INST}</s>','')
  response_mistral.append({'query':query,'answer':a})

In [ ]:
response_mistral

## Neural Hermes

In [ ]:
neural_hermes = AutoModelForCausalLM.from_pretrained("TheBloke/NeuralHermes-2.5-Mistral-7B-GGUF",model_file = "neuralhermes-2.5-mistral-7b.Q5_K_M.gguf", gpu_layers=400,context_length=4096)

In [ ]:
ans=[]
for inp in tqdm.tqdm(queries):
  a=predict(tool_data,query_data,inp,neural_hermes,0.5,'<|im_start|>system','<|im_end|>','<|im_start|>user','<|im_end|>','<|im_start>asisstant')
  ans.append({'query':query,'answer':a})

In [ ]:
inp

In [ ]:
ans

## LLama 7B

In [ ]:
llama = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-GGUF", model_file="llama-2-7b.Q4_K_M.gguf", model_type="llama",gpu_layers=400,context_length=8192,max_new_tokens=2048,temperature=0.5)

In [ ]:
ans=[]
for inp in tqdm.tqdm(queries):
  a=predict(tool_data,query_data,inp,llama,0.5,' ',' ',' ',' ','')
  ans.append({'query':inp,'answer':a})

In [ ]:
ans

##cybertron una

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install --no-cache-dir transformers sentencepiece